In [28]:
import pymssql
import pandas as pd
import numpy as np
import re
import time

In [89]:
#主函数测试

In [ ]:
#  最新版本 2021.4.4
def gettags(text,variety):
    word_lists = pd.read_csv('D:\Projects\新闻推荐\新闻推荐接口\get_tags\word_lists_72_20210404.csv')
    db = pymssql.connect('47.104.142.34', 'xueshu',   '@WSX3edc', 'AcademicExchangePlatform',charset='cp936')
    cursor = db.cursor()
    #LabelSort用不到
    sql = "select ObjectId,LabelSort,TagName,weight,relevancy_threshold from TB_Base_Tag where IsLastStage=1 order by LabelNum desc"
    cursor.execute(sql)
    wr = cursor.fetchall()  # 包含所有字段
    objectid=np.array(wr).T[0]
    labels= np.array(wr).T[2]
    weights = np.array(wr).T[3].astype(np.float)  # 矩阵转置并转换为float
    relevancy_threshold = np.array(wr).T[4].astype(np.float)
    text = text.replace("\n", "")
    text = text.replace(" ", "")
    text = text.replace("\u3000", "")
    n = []
    len_text = len(text)
    if len_text<800:
        pass
    elif len_text<1000:
        len_text = len_text ** (39/40)
    else:
        len_text=len_text ** (14/15) +211 #211约等于1000^(39/40)-1000^(14/15)
    words_freq = []
    for i in range(0, word_lists.shape[1]):
        freq = 0
        for word in word_lists.iloc[:, i]:
            if type(word) != str:
                continue
            if word.encode('utf-8').isalpha():  # 判断是否为纯英文，防止出现apple匹配pl的情况
                if text.find(word)==-1:
                    words_freq.append((word,0)) #改进算法速度，因为re正则匹配速度比较慢,太妙了
                else: 
                    regex = re.compile(r'[^A-Za-z](' + word + ')[^A-Za-z]')
                    word_freq = len(regex.findall(text))
                    if word_freq:
                        freq += 1
                    words_freq.append((word, word_freq))
            else:
                word_freq = text.count(word)
                if word_freq != 0:
                    freq += 1
                words_freq.append((word, word_freq))
        n.append((i, freq, freq / len_text))  # freq/len_text
    freq_sum = np.dot(np.array(n).T[1], weights)  #不同学科权重不同，感兴趣的学科freq_sum会有更大的权重，更容易相关
    k = freq_sum / len_text
    r = "相关" if k >= 0.33 else "不相关"
    #n.sort(key=lambda x: x[1], reverse=True) #各学科词表按匹配词数排序,应该按照相关性排！！！！！
    tags = []
    for t in n:
        correlated_rate = t[2] / relevancy_threshold[t[0]]  # 文章标签关联度
        if correlated_rate >= 1:  # 若比值大于阈值
            tags.append([objectid[t[0]], 100])  
        elif correlated_rate >= 0.5:
            tags.append([objectid[t[0]], correlated_rate * 100])
        else:
            continue
    words_freq = list(set(words_freq))  # 去除重复值
    words_freq.sort(key=lambda x: x[1], reverse=True)
    keywords = words_freq[0:5]  # 此表中匹配次数前五的词
    tags.sort(key=lambda x:x[1],reverse=True)
    db.close()
    cursor.close()
    if variety: #1为普通新闻 0为会议通知
        return r, tags[0:3], keywords
    else:
        for label_name in labels:
            if text.find(label_name)!=-1:
                for j in range(0,len(tags)):
                    if tags[j][0]==objectid[labels==label_name]:#布尔型索引,相同位置
                        tags[j][1]=100
                        break
                else:
                    tags.append([objectid[labels==label_name],100]) #for else语句 else会被break掉
        tags.sort(key=lambda x:x[1],reverse=True)
        r="相关"
        return r,tags[0:3],keywords    

In [121]:
def gettags_test(text,variety):
    word_lists = pd.read_csv('D:\Projects\新闻推荐\新闻推荐接口\get_tags\word_lists_72_20210404.csv')
    db = pymssql.connect('47.104.142.34', 'xueshu',   '@WSX3edc', 'AcademicExchangePlatform',charset='cp936')
    cursor = db.cursor()
    #LabelSort用不到
    sql = "select ObjectId,LabelSort,TagName,weight,relevancy_threshold from TB_Base_Tag where IsLastStage=1 order by LabelNum desc"
    cursor.execute(sql)
    wr = cursor.fetchall()  # 包含所有字段
    objectid=np.array(wr).T[0]
    labels= np.array(wr).T[2]
    weights = np.array(wr).T[3].astype(np.float)  # 矩阵转置并转换为float
    relevancy_threshold = np.array(wr).T[4].astype(np.float)
    text = text.replace("\n", "")
    text = text.replace(" ", "")
    text = text.replace("\u3000", "")
    n = []
    len_text = len(text)
    if len_text<800:
        pass
    elif len_text<1000:
        len_text = len_text ** (39/40)
    else:
        len_text=len_text ** (14/15) +211 #211约等于1000^(39/40)-1000^(14/15)
    words_freq = []
    for i in range(0, word_lists.shape[1]):
        freq = 0
        for word in word_lists.iloc[:, i]:
            if type(word) != str:
                continue
            if word.encode('utf-8').isalpha():  # 判断是否为纯英文，防止出现apple匹配pl的情况
                if text.find(word)==-1:
                    words_freq.append((word,0)) #改进算法速度，因为re正则匹配速度比较慢,太妙了
                else: 
                    regex = re.compile(r'[^A-Za-z](' + word + ')[^A-Za-z]')
                    word_freq = len(regex.findall(text))
                    if word_freq:
                        freq += 1
                    words_freq.append((word, word_freq))
            else:
                word_freq = text.count(word)
                if word_freq != 0:
                    freq += 1
                words_freq.append((word, word_freq))
        n.append((i, freq, freq / len_text))  # freq/len_text
    freq_sum = np.dot(np.array(n).T[1], weights)  #不同学科权重不同，感兴趣的学科freq_sum会有更大的权重，更容易相关
    k = freq_sum / len_text
    r = "相关" if k >= 0.33 else "不相关"
    #n.sort(key=lambda x: x[1], reverse=True) #各学科词表按匹配词数排序,应该按照相关性排！！！！！
    tags = []
    for t in n:
        correlated_rate = t[2] / relevancy_threshold[t[0]]  # 文章标签关联度
        if correlated_rate >= 1:  # 若比值大于阈值
            tags.append([labels[t[0]], 100])  #测试返回标签名！！！！！！！！
        elif correlated_rate >= 0.5:
            tags.append([labels[t[0]], correlated_rate * 100])
        else:
            continue
    words_freq = list(set(words_freq))  # 去除重复值
    words_freq.sort(key=lambda x: x[1], reverse=True)
    keywords = words_freq[0:5]  # 此表中匹配次数前五的词
    tags.sort(key=lambda x:x[1],reverse=True)
    db.close()
    cursor.close()
    if variety: #1为普通新闻 0为会议通知
        return r, tags[0:5], keywords
    else:
        for label_name in labels:
            if text.find(label_name)!=-1:
                for j in range(0,len(tags)):
                    if tags[j][0]==objectid[labels==label_name]:#布尔型索引,相同位置
                        tags[j][1]=100
                        break
                else:
                    tags.append([objectid[labels==label_name],100]) #for else语句 else会被break掉
        tags.sort(key=lambda x:x[1],reverse=True)
        r="相关"
        return r,tags[0:3],keywords

In [125]:
time_start=time.time()
text='''
宁波材料所构筑分子系统实现高效太阳光水解制氢

　　近日，中国科学院宁波材料技术与工程研究所先进能源材料工程实验室在太阳光水解制氢领域取得进展，相关成果以A molecular tandem cell for efficient solar water splitting 为题发表在《美国国家科学院院刊》上。

　　氢能是未来清洁能源的重要组成。受自然界光合作用启发，人工光合作用制氢相比其它高温制氢等手段更具有绿色、经济等特征。在人工光合作用中，阳光氧化水，并将氧化还原等价物转移至CO2，最终将CO2还原或者将质子还原为H2。该反应中，水氧化条件较为苛刻，因为水氧化会依次损失4个电子和产生4个质子，是一个动力学较为缓慢的过程。在电极表面上，光子吸收/电子转移启动步骤通常需要与催化剂整合，实现较短时间（微秒级）完成水氧化反应，从而避免电子与空穴复合。其中，比较主流的研究思路是采用半导体型核/壳结构光阳极材料，核壳结构可实现电子的高效转移来辅助光阳极材料、吸光基团和催化剂之间的局部电荷分离，使得光阳极水氧化效率大大提高。

　　虽然半导体材料在人工光合作用领域得到广泛研究和应用，但从设计的角度来看，分子组装方法在提高太阳能分解水电池的效率上更具有优势。然而，目前基于分子体系的太阳能制氢效率远低于半导体材料为基础的体系。

　　宁波材料所先进能源材料工程实验室研究员汪德高与美国北卡罗莱纳大学教堂山分校教授尤为、Thomas J. Meyer开展合作研究，报道了基于分子系统光电化学电池/光伏串联电池高效分解水的研究。该串联电池将染料敏化光电合成电池（DSPEC）加入有机光吸收基团，利用可见光将水转化为O2和H2。实验结果表明，可见光吸收电极的太阳能到氢能的转化效率大为改进，并为基于分子的太阳能燃料转化效率提供了基准。将DSPEC光电阳极与有机太阳能电池OSC结合，太阳能人工水分解制氢效率达到1.5％，相比之下，自然光合作用的效率仅为～1％。

　　该研究得到宁波市顶尖人才计划、浙江省领军型创新创业团队相关项目的支持。

　　论文链接



图1.左：OSC外部连接带有外部Pt阴极的DSPEC/PV串联设备；右：显示串联电池的能级设计和电子传输过程 



　　图2.a：串联装置分子系统产生气体过程的光电流信号；b：水分解过程中产生的H2和O2的定量示意图，红线和黑线对应于外部测量的H2和O2，黑色和红色虚线对应于随时间的积分光电流，显示从电流曲线产生的气体的理论值。




'''
text = text.replace("\n", "")
text = text.replace(" ", "")
text = text.replace("\u3000", "")
len_text = len(text)
print(len_text)
a=gettags_test(text,1)
time_end=time.time()
print(a)
print('totally cost',time_end-time_start)

1001
('相关', [['催化化学', 100], ['氢能', 100], ['物理化学', 98.34136223003176], ['精细化工', 84.7770364051998], ['有机化学', 81.38595494899181]], [('氢', 8), ('效率', 7), ('制氢', 6), ('氧', 6), ('太阳能', 6)])
totally cost 2.6554877758026123


d:\projects\新闻推荐\新闻推荐接口\venv\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()
d:\projects\新闻推荐\新闻推荐接口\venv\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if sys.path[0] == '':


In [93]:
#isactivity测试
#正则表达式测试
def isActivity(title):
    f=open('无关通讯过滤.txt','r',encoding='utf-8')
    lines=f.readlines()
    for word in [i.strip('\n') for i in lines]:
        regex = re.compile(word)
        if regex.search(title):
            f.close()
            return 1 #1为活动，国家级以下获奖类新闻
    f.close()
    return 0

In [92]:
t1 = "四川省能源中心获四川省牛人奖"
p1 = "\S\S{2,3}[省市州]\S+[奖]" 
t2="天大深圳学院绿色建筑与海绵城市科研中心部署今年工作"
p2="部署\S+工作"
t3="天津大学助力河南创新驱动发展 建设中原先进技术研究院"
p3="(组建|建设)\S+(实验室|研究院)"
regex = re.compile(p3)
result=regex.search(t3)
print(result.group(1))

建设


In [104]:
testdata=pd.read_excel('文章标题总结.xlsx',header=None)
for i in testdata.iterrows():
    print(i[0],isActivity(i[1][0]))

0 1
1 1
2 0
3 1
4 0
5 1
6 1
7 1
8 1
9 0
10 1
11 0
12 1
13 1
14 0
15 1
16 1
17 1
18 1
19 1
20 0
21 1
22 1
23 0
24 1
25 1
26 0
27 0
28 1
29 1
30 0
31 1
32 1
33 1


In [ ]:
#其他测试
import math
value=1000
value ** (1 / 1000)
math.log(100,1.0046157902783952) #log函数不行，因为一开始上涨很快，斜率
1000 ** (39/40)-(1000 ** (14/15))

In [12]:
#
#tag_weight和TB_base_tag整合
for i in range(0,72):
    weight=float(wr[i][3])
    relevancy_threshold=float(wr[i][4])
    objectid=wr[i][0]
    sql="update TB_Base_Tag set weight= %s,relevancy_threshold=%s where ObjectId=%s"
    cursor.execute(sql,(weight,relevancy_threshold,objectid))
    db.commit()

In [13]:
#老版本
def gettags(text):
    word_lists = pd.read_csv('D:\Projects\新闻推荐\新闻推荐接口\get_tags\word_lists_72.csv')
    db = pymssql.connect('47.104.142.34', 'xueshu',   '@WSX3edc', 'AcademicExchangePlatform')
    cursor = db.cursor()
    sql = "select * from Tag_weight"
    cursor.execute(sql)
    wr = cursor.fetchall()  # 包含所有字段
    weights = np.array(wr).T[3].astype(np.float)  # 矩阵转置并转换为float
    relevancy_threshold = np.array(wr).T[4].astype(np.float)
    text = text.replace("\n", "")
    text = text.replace(" ", "")
    text = text.replace("\u3000", "")
    n = []
    len_text = len(text)
    words_freq = []
    for i in range(0, word_lists.shape[1]):
        freq = 0
        for word in word_lists.iloc[:, i]:
            if type(word) != str:
                continue
            if word.encode('utf-8').isalpha():  # 判断是否为纯英文，防止出现apple匹配pl的情况
                regex = re.compile(r'[^A-Za-z](' + word + ')[^A-Za-z]')
                find = regex.findall(text)
                word_freq = len(find)
                if find:
                    freq += 1
                words_freq.append((word, word_freq))
            else:
                word_freq = text.count(word)
                if word_freq != 0:
                    freq += 1
                words_freq.append((word, word_freq))
        n.append((i, freq, freq / len_text))  # freq/len_text
    freq_sum = np.dot(np.array(n).T[1], weights)
    k = freq_sum / len_text
    r = "相关" if k >= 0.33 else "不相关"
    n.sort(key=lambda x: x[1], reverse=True)
    tags = []
    for i in n[0:3]:
        correlated_rate = i[2] / relevancy_threshold[i[0]]  # 文章标签关联度
        if correlated_rate >= 1:  # 若比值大于阈值
            tags.append((wr[i[0]][0], 100))
        elif correlated_rate >= 0.5:
            tags.append((wr[i[0]][0], correlated_rate * 100))
        else:
            continue
    words_freq = list(set(words_freq))  # 去除重复值
    words_freq.sort(key=lambda x: x[1], reverse=True)
    keywords = words_freq[0:5]  # 此表中匹配次数前五的词
    db.close()
    cursor.close()
    return r, tags, keywords


